<div class="alert alert-block alert-info">

# Imports

</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam, RMSprop
from keras.activations import relu, elu
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


<div class="alert alert-block alert-info">

# Dataframes

</div>

In [2]:
df_features = pd.read_csv('./dataset/ted_main_features.csv',sep="|",quotechar='"')

In [3]:
df_features.columns

Index(['comments', 'duration', 'languages', 'num_speaker', 'views',
       'film_month', 'film_dayofweek', 'published_month',
       'published_dayofweek', 'event_TED',
       ...
       'world', 'worldwide', 'worth', 'write', 'writer', 'wrong', 'ye', 'year',
       'york', 'young'],
      dtype='object', length=580)

In [4]:
df_features.shape

(2550, 580)

In [5]:
df_features.head()

,comments,duration,languages,num_speaker,views,film_month,film_dayofweek,published_month,published_dayofweek,event_TED,...,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,8,5,9,1,1,...,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,8,5,9,4,1,...,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,8,5,9,4,1,...,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,8,5,9,2,1,...,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,7,2,9,3,1,...,1,0,0,0,0,0,0,0,0,0


<div class="alert alert-block alert-info">

# Dummies

</div>

## Create Dummies

In [6]:
df_film_month = pd.get_dummies(df_features['film_month'],prefix='film_month',drop_first=True)
df_published_month = pd.get_dummies(df_features['published_month'],prefix='published_month',drop_first=True)

df_film_day = pd.get_dummies(df_features['film_dayofweek'],prefix='film_dayofweek',drop_first=True)
df_published_day = pd.get_dummies(df_features['published_dayofweek'],prefix='published_dayofweek',drop_first=True)

In [7]:
df_film_month.head(2)

,film_month_2,film_month_3,film_month_4,film_month_5,film_month_6,film_month_7,film_month_8,film_month_9,film_month_10,film_month_11,film_month_12
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0


In [8]:
df_film_day.head(2)

,film_dayofweek_1,film_dayofweek_2,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6
0,0,0,0,0,1,0
1,0,0,0,0,1,0


In [9]:
print(df_film_month.shape)
print(df_film_day.shape)

(2550, 11)
(2550, 6)


In [10]:
df_published_month.head(2)

,published_month_2,published_month_3,published_month_4,published_month_5,published_month_6,published_month_7,published_month_8,published_month_9,published_month_10,published_month_11,published_month_12
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0


In [11]:
df_published_day.head(2)

,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,1,0,0,0,0,0
1,0,0,0,1,0,0


In [12]:
print(df_published_month.shape)
print(df_published_day.shape)

(2550, 11)
(2550, 6)


## Drop Columns

In [13]:
df_features = df_features.drop(columns=['film_month','film_dayofweek','published_month','published_dayofweek'])
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,...,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Join Dummies

In [14]:
df_features = df_features.merge(df_film_month, left_index=True, right_index=True)
df_features = df_features.merge(df_published_month, left_index=True, right_index=True)
df_features = df_features.merge(df_film_day, left_index=True, right_index=True)
df_features = df_features.merge(df_published_day, left_index=True, right_index=True)
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,...,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,10,810,1,1,418368,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,8,723,4,1,707788,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,10,934,2,1,527314,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


<div class="alert alert-block alert-info">

# Train, Test, Split

</div>

In [15]:
# Generamos la matriz X y el vector y
X = df_features.drop(columns=['views'])
y = df_features['views']

In [16]:
# Separamos train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [17]:
# Estandarizamos las variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Chequeamos las shapes
print('X train shape: ', X_train.shape)
print('X test shape: ', X_test.shape)
print('y train shape: ', y_train.shape)
print('y test shape: ', y_test.shape)

X train shape:  (1785, 609)
X test shape:  (765, 609)
y train shape:  (1785,)
y test shape:  (765,)


In [18]:
# Primero, definimos una función que permita construir el modelo, donde todos los valores de los hiperparámetros se obtienen del diccionario params 

def build_model(x_train):

    model = Sequential()
    
    model.add(Dense(512, input_dim=x_train.shape[1]))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    model.compile(optimizer=Adam(lr=3e-6), loss='mse', metrics=['mae'])

    return model

In [19]:
model = build_model(X_train)

In [20]:
# Instanciamos nuestro objeto earlystopping y definimos nuestra lista de callbacks
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.03, patience = 4, verbose = 1, restore_best_weights=True)
callbacks_list = [early_stopping]

In [21]:
history = model.fit(X_train, y_train, batch_size=16, epochs=8000, validation_split = 0.2, verbose=1)

Train on 1428 samples, validate on 357 samples
Epoch 1/8000
1428/1428 [==============================] - 2s 2ms/step - loss: 8484821969945.8154 - mean_absolute_error: 1650201.2626 - val_loss: 9478500153831.6191 - val_mean_absolute_error: 1721001.5497
Epoch 2/8000
1428/1428 [==============================] - 1s 359us/step - loss: 8484822077887.4619 - mean_absolute_error: 1650201.2605 - val_loss: 9478500153831.6191 - val_mean_absolute_error: 1721001.5497
Epoch 3/8000
1428/1428 [==============================] - 0s 349us/step - loss: 8484822021530.1729 - mean_absolute_error: 1650201.2542 - val_loss: 9478500153831.6191 - val_mean_absolute_error: 1721001.5497
Epoch 4/8000
1428/1428 [==============================] - 1s 356us/step - loss: 8484821898719.0137 - mean_absolute_error: 1650201.2633 - val_loss: 9478500153831.6191 - val_mean_absolute_error: 1721001.5497
Epoch 5/8000
1428/1428 [==============================] - 1s 356us/step - loss: 8484822036032.5381 - mean_absolute_error: 1650201.2

1428/1428 [==============================] - 1s 354us/step - loss: 8484820685660.5039 - mean_absolute_error: 1650200.8416 - val_loss: 9478498257876.9746 - val_mean_absolute_error: 1721000.9835
Epoch 41/8000
1428/1428 [==============================] - 0s 348us/step - loss: 8484820507960.6494 - mean_absolute_error: 1650200.7917 - val_loss: 9478497394710.9473 - val_mean_absolute_error: 1721000.8596
Epoch 42/8000
1428/1428 [==============================] - 1s 353us/step - loss: 8484820183401.4121 - mean_absolute_error: 1650200.7227 - val_loss: 9478497194982.1855 - val_mean_absolute_error: 1721000.7756
Epoch 43/8000
1428/1428 [==============================] - 1s 354us/step - loss: 8484819778069.5127 - mean_absolute_error: 1650200.6201 - val_loss: 9478496972123.0703 - val_mean_absolute_error: 1721000.6758
Epoch 44/8000
1428/1428 [==============================] - 0s 348us/step - loss: 8484819773663.7305 - mean_absolute_error: 1650200.5508 - val_loss: 9478496686481.5684 - val_mean_absolute

Epoch 80/8000
1428/1428 [==============================] - 1s 354us/step - loss: 8484375483274.3984 - mean_absolute_error: 1650078.6534 - val_loss: 9478010655718.1855 - val_mean_absolute_error: 1720871.6779
Epoch 81/8000
1428/1428 [==============================] - 1s 351us/step - loss: 8484309033258.3086 - mean_absolute_error: 1650062.5550 - val_loss: 9477950285129.8594 - val_mean_absolute_error: 1720855.9244
Epoch 82/8000
1428/1428 [==============================] - 1s 353us/step - loss: 8484265324784.9414 - mean_absolute_error: 1650047.2770 - val_loss: 9477886997547.0254 - val_mean_absolute_error: 1720839.3687
Epoch 83/8000
1428/1428 [==============================] - 1s 364us/step - loss: 8484207099077.9160 - mean_absolute_error: 1650027.9426 - val_loss: 9477816396734.0273 - val_mean_absolute_error: 1720821.0315
Epoch 84/8000
1428/1428 [==============================] - 1s 352us/step - loss: 8484113900277.2432 - mean_absolute_error: 1650008.7976 - val_loss: 9477738731984.6719 - val

1428/1428 [==============================] - 0s 347us/step - loss: 8468796130610.9131 - mean_absolute_error: 1646095.2836 - val_loss: 9462116286005.0645 - val_mean_absolute_error: 1716962.9275
Epoch 120/8000
1428/1428 [==============================] - 1s 353us/step - loss: 8467409522860.1006 - mean_absolute_error: 1645693.6471 - val_loss: 9460833453133.4453 - val_mean_absolute_error: 1716655.7773
Epoch 121/8000
1428/1428 [==============================] - 0s 349us/step - loss: 8467011518509.8936 - mean_absolute_error: 1645493.6264 - val_loss: 9459555742717.1309 - val_mean_absolute_error: 1716349.0480
Epoch 122/8000
1428/1428 [==============================] - 0s 348us/step - loss: 8464982667504.9414 - mean_absolute_error: 1645071.3305 - val_loss: 9458129726352.1348 - val_mean_absolute_error: 1716008.6548
Epoch 123/8000
1428/1428 [==============================] - 0s 347us/step - loss: 8462739364717.7139 - mean_absolute_error: 1644667.7405 - val_loss: 9456572998526.9238 - val_mean_abso

 336/1428 [======>.......................] - ETA: 0s - loss: 12230856159427.0469 - mean_absolute_error: 1623737.9464

KeyboardInterrupt: 